In [ ]:
import numpy as np
from liberate import fhe
from liberate.fhe import presets

params = presets.params["gold"]
print(params)
params["devices"] = [0]
# params["scale_bits"] = 49
engine = fhe.ckks_engine(verbose=True, **params)
target_N = 10**7 # 10,000,000
required_num_ct = round(target_N/engine.num_slots) #32768

print(required_num_ct)
print(engine.num_slots)
# generate Keys

sk = engine.create_secret_key()
pk = engine.create_public_key(sk)
evk = engine.create_evk(sk)
gk = engine.create_galois_key(sk)

In [ ]:
# 02. Mean
# generate Data

cts = []
ms = []
sample_range = 1
for _ in range(required_num_ct):
    m = engine.example(-sample_range, sample_range).real
    ct = engine.encorypt(m, pk, level=engine.num_levels - 4)
    # ct = engine.encorypt(m, pk, level=0)
    ms.append(m)
    cts.append(ct)
# cacle mean
    
result = engine.add(cts[0], cts[1])

for ct in cts[2:]:
    result = engine.add(result, ct)

ct_mean = engine.mean(result, gk, alpha=required_num_ct)

calculated_mean = engine.decrode(ct_mean, sk).mean()
target_mean = np.array(ms).mean()

print("====="*20)
print(f">>\tcalc mean\t: {calculated_mean.real:20.13e}")
print(f">>\ttarget mean\t: {target_mean:20.13e}")
print(f">>\terror\t\t: {(calculated_mean - target_mean).real:20.13e}")

# 03. Var(iance)
# generate data

cts = []
ms = []
t = 1

for _ in range(required_num_ct):
    m = engine.example(-t, t).real
#     ct = engine.encorypt(m, pk, level=engine.num_levels - 8)
    ct = engine.encorypt(m, pk, level=0)
    ms.append(m)
    cts.append(ct)




In [ ]:
%%time

# calc Var 

result = engine.add(cts[0], cts[1])

for ct in cts[2:]:
    result = engine.add(result, ct)

ct_mean = engine.mean(result, gk, alpha=required_num_ct)

devs = []

# for idx, ct in enumerate(cts):
#     cts[idx] = engine.level_up(ct, ct_mean.level)

for ct in cts:
    dev = engine.sub(ct, ct_mean)

    dev = engine.square(dev, evk, relin=False)
    devs.append(dev)


result = engine.cc_add(devs[0], devs[1])

for dev in devs[2:]:
    result = engine.cc_add(dev, result)

result = engine.relinearize(result, evk)

ct_var = engine.mean(result, gk, alpha=required_num_ct)

calculated_var = engine.decrode(ct_var, sk).mean()
target_var = np.array(ms).real.var() + np.array(ms).imag.var() * 1j

print(calculated_var, target_var, calculated_var - target_var)
print(f">>\tcalc var\t: {calculated_var.real:20.13e}")
print(f">>\ttarget var\t: {target_var.real:20.13e}")
print(f">>\terror\t\t: {(calculated_var - target_var).real:20.13e}")

# 04. Standard Deviation(STD)
# generate data

cts = []
ms = []
t = 1

for _ in range(required_num_ct):
    m = engine.example(-t, t).real
#     ct = engine.encorypt(m, pk, level=engine.num_levels - 8)
    ct = engine.encorypt(m, pk, level=0)
    ms.append(m)
    cts.append(ct)

In [ ]:
%%time

##############
#### mean ####
##############

result = engine.add(cts[0], cts[1])

for ct in cts[2:]:
    result = engine.add(result, ct)

ct_mean = engine.mean(result, gk, alpha=required_num_ct)

devs = []

#############
#### var ####
#############
# for idx, ct in enumerate(cts):
#     cts[idx] = engine.level_up(ct, ct_mean.level)

for ct in cts:
    dev = engine.sub(ct, ct_mean)

    dev = engine.square(dev, evk, False)
    devs.append(dev)


result = engine.cc_add(devs[0], devs[1])

for dev in devs[2:]:
    result = engine.cc_add(dev, result)

result = engine.relinearize(result, evk)


ct_var = engine.mean(result, gk, alpha=required_num_ct)

calculated_var = engine.decrode(ct_var, sk).mean()
target_var = np.array(ms).real.var() + np.array(ms).imag.var() * 1j

print(calculated_var, target_var, calculated_var - target_var)
#############
#### std ####
#############

ct_std = engine.sqrt(ct_var, evk=evk)
print(f"""{engine.decrode(ct_std, sk)[0].real:10.6f} | {np.std(ms):10.6f}  |  {engine.decrode(ct_std, sk)[0].real-np.std(ms):20.15f}""")
print(f">>\tcalc var\t: {calculated_var.real:20.13e}")
print(f">>\ttarget var\t: {target_var.real:20.13e}")
print(f">>\terror\t\t: {(calculated_var - target_var).real:20.13e}")
calculated_std = engine.decrode(ct_std, sk)
calculated_std[0].real
